In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
x = [[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]] # [공부 시간, 과외 시간], 7행 2열
print(x)
print(type(x))

xData = np.array(x)
print(xData)
print(type(xData))

[[2, 3], [4, 3], [6, 4], [8, 6], [10, 7], [12, 8], [14, 9]]
<class 'list'>
[[ 2  3]
 [ 4  3]
 [ 6  4]
 [ 8  6]
 [10  7]
 [12  8]
 [14  9]]
<class 'numpy.ndarray'>


In [3]:
y = [0, 0, 0, 1, 1, 1, 1] # [합격 여부], 실제값, 1행 7열
print(y)
print(type(y))

yData = np.array(y)
print(yData)
print(type(yData))

# reshape(): 넘파이에서 데이터는 그대로 유지한채 배열의 형태(차원)를 변경한다.
# 1행 7열 넘파이 배열 => 7행 1열 넘파이 배열
yData = np.array(y).reshape(7, 1)
print(yData)
print(type(yData))

[0, 0, 0, 1, 1, 1, 1]
<class 'list'>
[0 0 0 1 1 1 1]
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
<class 'numpy.ndarray'>


xData와 yData를 저장할 placeholder를 만든다.

In [4]:
# placeholder에 넘파이 배열을 대입하는 경우 shape 속성을 이용해서 대입될 넘파이 배열의 차원을 지정해야 한다.
# [None, 2] => placeholder에 대입될 넘파이 배열의 행의 개수는 몇개라도 상관없고 열의 개수는 무조건 2개라는 의미이다.
X = tf.placeholder(dtype=tf.float32, shape=[None, 2]) # xData가 대입될 placeholder
Y = tf.placeholder(dtype=tf.float32, shape=[None, 1]) # yData가 대입될 placeholder

가중치(a)와 y절편(b)을 임의의 값으로 정한다.

In [5]:
a = tf.Variable(tf.random_uniform([2, 1], dtype=tf.float32)) # 행렬의 내적을 이용하기 위해서 난수를 2행 1열로 발생시긴다.
b = tf.Variable(tf.random_uniform([1], dtype=tf.float32))
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print('a\n', sess.run(a), sep='')
print('b\n', sess.run(b), sep='')
print('a1: %f, a2: %f, b: %f' % (sess.run(a)[0], sess.run(a)[1], sess.run(b)))
print('a1: {:f}, a2: {:f}, b: {:f}'.format(sess.run(a)[0][0], sess.run(a)[1][0], sess.run(b)[0]))

a
[[0.738878  ]
 [0.01253402]]
b
[0.90526175]
a1: 0.738878, a2: 0.012534, b: 0.905262
a1: 0.738878, a2: 0.012534, b: 0.905262


시그모이드 방정식, 오차 함수, 경사 하강법

In [6]:
# Y = 1 / (1 + np.e ** -(a1 * x1 + a2 * x2 + b))
# sigmoid() 메소드로 tensorflow에서 시그모이드 방정식을 계산한다.
# matmul() 메소드로 tensorflow에서 행렬의 내적을 계산한다.
y = tf.sigmoid(tf.matmul(X, a) + b) # 시그모이드 방정식, 예측값

# -tf.reduce_mean(실제값 * tf.log(예측값) + (1 - 실제값) * tf.log(1 - 예측값))
loss = -tf.reduce_mean(Y * tf.log(y) + (1 - Y) * tf.log(1 - y)) # 오차 함수

gradient_descent = tf.train.GradientDescentOptimizer(0.1).minimize(loss) # 경사 하강법

sigmoid() 메소드를 실행한 결과로 예측값(0 또는 1)을 계산한다.

In [7]:
# sigmoid() 메소드의 실행 결과가 0.5 이상이면 1을 0.5 미만이면 0을 리턴시킨다.
sess = tf.Session()
# cast(data, dtype) 메소드는 tensorflow에서 데이터 형변환을 한다.
predict = tf.cast(tf.constant([1.9, 2.3]), dtype=tf.int32)
print(sess.run(predict))
# cast() 메소드는 캐스팅할 데이터가 boolean 타입일 경우 True는 1로 False는 0으로 형변환 한다.
predict = tf.cast(tf.constant([True, False]), dtype=tf.int32)
print(sess.run(predict))
predict = tf.cast([0.5 >= 0.5, 0.5 < 0.5], dtype=tf.int32)
print(sess.run(predict))

[1 2]
[1 0]
[1 0]


In [8]:
# 시그모이드 함수의 실행 결과(예측값, y)로 최종 예측값을 계산한다.
predict = tf.cast(y >= 0.5, dtype=tf.float32) # 최종 예측값

최종 예측값(predict)과 실제값(Y)이 일치하는 정도(정확도, accuracy)를 계산한다.

In [9]:
sess = tf.Session()
# equal() 메소드는 tensorflow에서 인수로 지정한 두 값이 같으면 True, 다르면 False를 리턴한다.
print(sess.run(tf.equal(1, 1)), sess.run(tf.equal(1, 0)))
# 정확도는 equal() 메소드로 예측값과 실제값이 같은가 비교 후 리턴되는 True 또는 False를 cast() 
# 메소드로 1 또는 0으로 캐스팅하고 평균을 계산한다.
accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, Y), dtype=tf.float32))

True False


학습 시킨다.

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(3001):
    # run() 메소드로 여러개를 한 번에 실행하려면 run() 메소드로 실행할 목록을 리스트로 묶어서 전달한다.
    # run([실행할 내용, ...], feed_dict={플레이스 홀더 이름: 플레이스 홀더에 저장할 데이터, ...})
    _, loss_, a_, b_, predict_, accuracy_ = sess.run([gradient_descent, loss, a, b, predict, accuracy], feed_dict={X: xData, Y: yData})
    if epoch % 300 == 0:
        print('epoch: {:4d}, loss: {:7.4f}, a1: {:7.4f}, a2: {:7.4f}, b: {:8.4f}, accuracy: {:8.4f}'.format(
            epoch, loss_, a_[0][0], a_[1][0], b_[0], accuracy_
        ))
# ===== 학습 완료

print('실제값\n', yData, sep='')
print('최종 예상값\n', predict_, sep='')
print('정확도: {:6.2%}'.format(accuracy_), sep='')

epoch:    0, loss:  2.0280, a1:  0.4475, a2:  0.4333, b:   0.2917, accuracy:   0.5714
epoch:  300, loss:  0.2611, a1:  0.7942, a2: -0.5073, b:  -2.5235, accuracy:   0.8571
epoch:  600, loss:  0.1882, a1:  0.7972, a2: -0.2443, b:  -3.9671, accuracy:   1.0000
epoch:  900, loss:  0.1476, a1:  0.7124, a2:  0.0734, b:  -5.0161, accuracy:   1.0000
epoch: 1200, loss:  0.1212, a1:  0.6133, a2:  0.3718, b:  -5.8481, accuracy:   1.0000
epoch: 1500, loss:  0.1025, a1:  0.5179, a2:  0.6387, b:  -6.5394, accuracy:   1.0000
epoch: 1800, loss:  0.0887, a1:  0.4312, a2:  0.8749, b:  -7.1313, accuracy:   1.0000
epoch: 2100, loss:  0.0781, a1:  0.3538, a2:  1.0840, b:  -7.6490, accuracy:   1.0000
epoch: 2400, loss:  0.0697, a1:  0.2852, a2:  1.2698, b:  -8.1092, accuracy:   1.0000
epoch: 2700, loss:  0.0629, a1:  0.2242, a2:  1.4361, b:  -8.5236, accuracy:   1.0000
epoch: 3000, loss:  0.0573, a1:  0.1699, a2:  1.5860, b:  -8.9003, accuracy:   1.0000
실제값
[[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]]
최종 예상값
[[0.]
 

In [11]:
# d = np.array([[5, 8]])
d = np.array([5, 8]).reshape(1, 2)
print('합격' if sess.run(predict, feed_dict={X: d})[0][0] == 1.0 else '불합격')

합격
